In [ ]:
!pip install transformers
!pip install torch
!pip install datasets

In [ ]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
import json
import torch
from transformers import AutoTokenizer
from transformers import DefaultDataCollator
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
folder_path = '/content/drive/MyDrive/model'
import os
os.listdir(folder_path)

Load dataset

In [ ]:
def read_data(data_dir):
    with open(data_dir, 'r', encoding = 'utf-8') as f:
        data = json.load(f)
    return data

Convert file json to Dataframe

In [ ]:
dataset = read_data( folder_path + '/Dataset/UIT-ViQuAD.json')
contexts = []
questions = []
answers = []
ids = []
for data in dataset['data']:
    for para in data['paragraphs']:
        context = para['context']
        for qa in para['qas']:
            question = qa['question']
            for answer in qa['answers']:
                contexts.append(context)
                questions.append(question)
                answers.append({'answer_start': [answer['answer_start']], 'text': [answer['text']]})
                ids.append(qa['id'])
icqa = [ids, contexts, questions, answers] 

In [ ]:
df = pd.DataFrame(list(zip(ids, contexts, questions, answers)),
               columns =['ids', 'contexts', 'questions', 'answers'])
df.head()

Separate train/test

In [ ]:
train=df.sample(frac=0.8,random_state=200)
test=df.drop(train.index)

Function pre process-tokenizer

In [ ]:
def preprocess_function(examples):
    # examples = train[input_ids]
    questions = [q.strip() for q in examples["questions"]]

    inputs = tokenizer(
        questions,
        examples["contexts"],
        max_length=512,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
import pyarrow as pa
import datasets

table = pa.Table.from_pydict(train)
rows = table.to_pydict()
dataset_train = datasets.Dataset.from_dict(rows)

table2 = pa.Table.from_pydict(test)
rows2 = table.to_pydict()
dataset_eval = datasets.Dataset.from_dict(rows2)

In [ ]:
tokenized_squad_eval = dataset_eval.map(preprocess_function, batched=True)
tokenized_squad_train = dataset_train.map(preprocess_function, batched=True)

In [ ]:
data_collator = DefaultDataCollator()
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

In [ ]:
print(tokenized_squad_train)

In [ ]:
print(tokenized_squad_train[0])

In [ ]:
training_args = TrainingArguments(
    output_dir="transformer_qa",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    num_train_epochs=2,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad_train,
    eval_dataset=tokenized_squad_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

In [ ]:
question = '''Giai đoạn năm 1955-1976, Phạm Văn Đồng nắm giữ chức vụ gì?'''
context = '''Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4 năm 2000) là Thủ tướng đầu tiên của nước Cộng hòa Xã hội chủ nghĩa Việt Nam từ năm 1976 (từ năm 1981 gọi là Chủ tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa từ năm 1955 đến năm 1976. Ông là vị Thủ tướng Việt Nam tại vị lâu nhất (1955–1987). Ông là học trò, cộng sự của Chủ tịch Hồ Chí Minh. Ông có tên gọi thân mật là Tô, đây từng là bí danh của ông. Ông còn có tên gọi là Lâm Bá Kiệt khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm (Chủ nhiệm là Hồ Học Lãm).'''

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="/content/drive/MyDrive/model/transformer_qa/checkpoint-500")
question_answerer(question=question, context=context)